In [1]:
import cassandra

In [2]:
from cassandra.cluster import Cluster

In [3]:
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

Test our connection by doing a select * on a table that we have not created yet:

In [4]:
try:
    session.execute("""select * from music_library""")
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


Let's create a keyspace to do our work in:

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """
    )
    
except Exception as e:
    print(e)

Creating a music library of albums: Each album will have album name, artist name, and year

### However, 

Since Apache Cassandra is a NoSQL Database, we can't model our data and create our table without more information. 

### What queries will I be performing on this data?

In our case, we would like to be able to get every album that was released in a particular year:

```select * from music_library WHERE year=1970```

Because of this I need to be able to do a ```WHERE``` on ```year```. ```year``` will become my partition key, and artist name will be my clustering column to make each Primary Key unique. Remember there are no duplicates in Apache Cassandra. 

Table Name: music_library<br>
column 1: Album Name<br>
column 2: Artist Name<br>
column 3: Year<br>
PRIMARY KEY(year, artist name)

### Now to translate this information into a Create Table Statement.

In [8]:
query = "CREATE TABLE IF NOT EXISTS udacity.music_library "
query = query + "(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

Check to ensure that our table was created:

In [10]:
query = "select count(*) from udacity.music_library"
try:
    count = session.execute(query)
except Exception as e:
    print(e)
    
print(count.one())

Row(count=0)


Let's insert 2 rows:

In [12]:
query = "INSERT INTO udacity.music_library (year, artist_name, album_name)"
query = query + " VALUES (%s, %s, %s)"

try:
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)

Validate that our data was inserted into our table:

In [13]:
query = 'SELECT * FROM udacity.music_library'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.year, row.album_name, row.artist_name)

1965 Rubber Soul The Beatles
1970 Let it Be The Beatles


Drop the table:

In [15]:
query = "drop table udacity.music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Close the session:

In [16]:
session.shutdown()
cluster.shutdown()